In [ ]:
import pandas as pd
import datetime as dt
import time
from os import listdir
from os.path import isfile, join
import numpy as np

In [ ]:
folder = "./gateio_data/major_trade_data"
files = [f for f in listdir(folder) if isfile(join(folder, f))]

In [ ]:
#files = [i for i in files if i[6] in ['202307', '202308', '202309']]


In [ ]:
def date_processing(df):
    df = df[["timestamp","dealid","price","amount","side"]]
    df['hour'] = [i[:-3] for i in df['timestamp']]#.apply(lambda x: x.strftime('%Y-%m-%d-%H'))
    df.set_index("hour", inplace=True)
    df.sort_values(by='timestamp', inplace=True)
    return df

def vwap1():
    df = pd.DataFrame()
    for each_file in files[:1]:
            if each_file != ".DS_Store":
                #vwap calc based on https://school.stockcharts.com/doku.php?id=technical_indicators:vwap_intraday
                tp = (each_file['price'].groupby(level=0).max()+each_file['price'].groupby(level=0).min()
                    +each_file['price'].groupby(level=0).tail(1))/3
                v = each_file['amount'].groupby(level=0).sum()
                tp_v = tp*v
                tp_v_cum = tp_v.cumsum()
                v_cum = v.cumsum()
                vwap = tp_v_cum/v_cum
                new_df = pd.DataFrame(vwap)
                new_df['ticker'] = each_file[:-7]
                new_df.columns = ['vwap', 'ticker']
                if df.empty:
                    df = new_df
                else: 
                    df = pd.concat([df, new_df])
    #df.to_csv("./data/aggregated_trade_data.csv")
    return df

def vwap_vanilla(df):
    q = df.amount.values
    p = df.price.values
    return df.assign(vwap=(p * q).cumsum() / q.cumsum())

def vwap2(df):
    df = df.groupby(df.index.date, group_keys=False).apply(vwap_vanilla)
    return df

In [ ]:
def each_assetmonth_sort(each_assetmonth):
    #read and sort each month of data
    df = pd.read_csv(folder+"/"+each_assetmonth)
    df = df[["timestamp","dealid","price","amount","side"]]
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d-%H-%M')
    df.sort_values('timestamp', inplace=True)
    df.set_index('timestamp', inplace=True)
    #calculate vwap
    df = vwap2(df)
    #assemble all assets and times together
    df = df[['price', 'vwap']]
    #df['vwap_pct_change'] = df['vwap'].pct_change()
    #df['price_pct_change'] = df['price'].pct_change()
    df['ticker'] = each_assetmonth[:-7]
    return df

batched_data =[]
for files in 
for each_assetmonth in files:
        if each_assetmonth != ".DS_Store":
              batched_data.append(each_assetmonth_sort(each_assetmonth))

            


# [i.split('-')[0] for i in files]
# matching_files = [i for i in listdir(folder) if isfile(join(folder,i)) and \
#          'SFP_USDT' in i]
# matching_files

In [ ]:
agg_df = pd.concat(batched_data)

In [ ]:
agg_df.head()

In [ ]:
agg_df.to_csv("./gateio_data/majors_agg_trade_data.csv")

In [ ]:
#specific processing
#agg_df = pd.read_csv("/Users/aronnyberg/code/quanty2/gateio_data/btc_trade_data.csv")


In [ ]:
# agg_df = agg_df[agg_df['ticker'] =="BTC_USDT"]
agg_df['timestamp'] = pd.to_datetime(agg_df['timestamp'])
agg_df.set_index('timestamp', inplace=True)
agg_df = agg_df['vwap']

In [ ]:
batch_size = 100000  # Number of data points per batch

# Calculate the number of batches
num_batches = len(agg_df) // batch_size

# Initialize an empty list to store the batched data
batched_data = []

# Loop through the data and create batches
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    batch = agg_df.iloc[start_idx:end_idx]
    # Here, you can perform any desired aggregation on the batch, e.g., sum, mean, etc.
    batched_data.append(batch.pct_change().resample('T').sum())

In [ ]:
# Concatenate the batched data into a single DataFrame
#daily_btc_rets = pd.concat(batched_data)
daily_rets = pd.concat(batched_data)

In [ ]:
daily_rets = daily_rets.resample('T').sum()

In [ ]:
daily_rets.cumsum().plot()

In [ ]:
pd.concat(batched_data[:10])